In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sarlren/isic16")

print("Path to dataset files:", path)

100%|██████████| 832M/832M [00:13<00:00, 66.8MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/sarlren/isic16/versions/2


In [2]:
!git clone https://github.com/WuJunde/MedSegDiff.git
!mv MedSegDiff/* ./

Cloning into 'MedSegDiff'...
remote: Enumerating objects: 716, done.
remote: Counting objects: 100% (397/397), done.
remote: Compressing objects: 100% (149/149), done.
remote: Total 716 (delta 340), reused 248 (delta 248), pack-reused 319 (from 2)
Receiving objects: 100% (716/716), 3.90 MiB | 28.56 MiB/s, done.
Resolving deltas: 100% (396/396), done.


In [3]:
!mkdir /kaggle

mkdir: cannot create directory ‘/kaggle’: File exists


In [4]:
!mkdir /kaggle/working

In [5]:
!mkdir /kaggle/working/out

In [6]:
!mkdir /kaggle/working/isic

In [7]:
!cp -r /content/guided_diffusion /content/scripts/guided_diffusion

In [8]:
import subprocess
subprocess.call("cp -r "+path+"/ISBI2016_ISIC_Part1_Test_Data/* /kaggle/working/isic/", shell=True)
subprocess.call("cp -r "+path+"/ISBI2016_ISIC_Part1_Training_Data/* /kaggle/working/isic/", shell=True)
subprocess.call("cp -r "+path+"/ISBI2016_ISIC_Part1_Training_GroundTruth/* /kaggle/working/isic/", shell=True)
subprocess.call("cp -r "+path+"/ISBI2016_ISIC_Part1_Test_GroundTruth/* /kaggle/working/isic/", shell=True)
subprocess.call("cp "+path+"/ISBI2016_ISIC_Part3B_Test_GroundTruth.csv /kaggle/working/isic/", shell=True)
subprocess.call("cp "+path+"/ISBI2016_ISIC_Part3B_Training_GroundTruth.csv /kaggle/working/isic/", shell=True)

0

In [9]:
!pip install blobfile
!pip install nibabel
!pip install visdom
!pip install torchsummary
!pip install mpi4py
!pip install batchgenerators

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 653.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408195 sha256=34c6a9061bc0eed665a737b7a6aa660f78b44f0b8bf0313ac88bc86c5435138c
  Stored in directory: /root/.cache/pip/wheels/fa/a4/bb/2be445c295d88a74f9c0a4232f04860ca489a5c7c57eb959d9
Successfully built visdom
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.3/466.3 kB 1.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-4.0.3-cp311-cp311-linux_x86_64.whl size=4458267 sha256=1b7c003d56809ef81e75242d1f61da76c886adf1ca61ab1fd31a0bbb2d946692
  Stored in direc

In [1]:
import pdb

In [5]:
import sys
import os
sys.path.append("/content/scripts")
sys.path.append("./")
from ssl import OP_NO_TLSv1
import nibabel as nib
import sys
import random
import numpy as np
import time
from PIL import Image
import torch.distributed as dist
import torchvision.utils as vutils
from guided_diffusion import dist_util, logger
from guided_diffusion.resample import create_named_schedule_sampler
from guided_diffusion.bratsloader import BRATSDataset, BRATSDataset3D
from guided_diffusion.isicloader import ISICDataset
from guided_diffusion.custom_dataset_loader import CustomDataset
from guided_diffusion.script_util import (
    NUM_CLASSES,
    model_and_diffusion_defaults,
    create_model_and_diffusion,
    args_to_dict,
    add_dict_to_argparser,
)
import torch as th
from guided_diffusion.train_util import TrainLoop
from guided_diffusion.utils import staple
import torchvision.transforms as transforms
from torchsummary import summary

from dataclasses import dataclass

@dataclass
class Args:
  data_name: str
  data_dir: str
  out_dir: str
  image_size: int
  num_channels: int
  class_cond: bool
  num_res_blocks: int
  num_heads: int
  learn_sigma: bool
  use_scale_shift_norm: bool
  attention_resolutions: str
  diffusion_steps: int
  noise_schedule: str
  rescale_learned_sigmas: bool
  rescale_timesteps: bool
  lr: float
  batch_size: int
  patience: int
  min_delta: float
  microbatch: int
  ema_rate: str
  log_interval: int
  save_interval: int
  resume_checkpoint: str
  schedule_sampler: str
  weight_decay: float
  lr_anneal_steps: int
  use_fp16: bool
  fp16_scale_growth: float
  gpu_dev: str
  multi_gpu: str
  in_ch: int
  num_heads_upsample: int
  num_head_channels: int
  resblock_updown: bool
  dpm_solver: bool
  version: str
  channel_mult: str
  dropout: float
  use_checkpoint: bool
  use_new_attention_order: bool
  timestep_respacing: str
  use_kl: bool
  predict_xstart: bool
  model_path: str
  num_ensemble: int
  use_ddim: bool
  clip_denoised: bool

argsdict = dict(
  data_name = 'ISIC',
  data_dir = "/kaggle/working/isic",
  out_dir = "/kaggle/working/out",
  image_size = 256,
  num_channels = 128,
  class_cond = False,
  num_res_blocks = 2,
  num_heads = 1,
  learn_sigma = True,
  use_scale_shift_norm = False,
  attention_resolutions = "16",
  diffusion_steps = 1000,
  noise_schedule = 'linear',
  rescale_learned_sigmas = False,
  rescale_timesteps = False,
  lr = 1e-4,
  batch_size = 4,
  patience = 2,
  min_delta = 0.001,
  microbatch = -1,  # -1 disables microbatches
  ema_rate = "0.9999",  # comma-separated list of EMA values
  log_interval = 100,
  save_interval = 5000,
  resume_checkpoint = None, #"/results/pretrainedmodel.pt"
  schedule_sampler="uniform",
  weight_decay=0.0,
  lr_anneal_steps=400, # used for early stopping
  use_fp16=False,
  fp16_scale_growth=1e-3,
  gpu_dev = "0",
  multi_gpu = None, #"0,1,2"

  # useful for sampling only:
  model_path = "/kaggle/working/out/savedmodel000400.pt",
  num_ensemble = 5,
  use_ddim = False, #originally False
  clip_denoised = True,
)
argsdict.update(model_and_diffusion_defaults())

args = Args(**argsdict)

seed=10
th.manual_seed(seed)
th.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)

In [8]:
def train():
    dist_util.setup_dist(args)
    logger.configure(dir = args.out_dir)

    logger.log("creating data loader...")

    if args.data_name == 'ISIC':
        tran_list = [transforms.Resize((args.image_size,args.image_size)), transforms.ToTensor(),]
        transform_train = transforms.Compose(tran_list)

        ds = ISICDataset(args, args.data_dir, transform_train)
        args.in_ch = 4
    elif args.data_name == 'BRATS':
        tran_list = [transforms.Resize((args.image_size,args.image_size)),]
        transform_train = transforms.Compose(tran_list)

        ds = BRATSDataset3D(args.data_dir, transform_train, test_flag=False)
        args.in_ch = 5
    else :
        tran_list = [transforms.Resize((args.image_size,args.image_size)), transforms.ToTensor(),]
        transform_train = transforms.Compose(tran_list)
        print("Your current directory : ",args.data_dir)
        ds = CustomDataset(args, args.data_dir, transform_train)
        args.in_ch = 4

    datal= th.utils.data.DataLoader(
        ds,
        batch_size=args.batch_size,
        shuffle=True)
    data = iter(datal)

    logger.log("creating model and diffusion...")

    model, diffusion = create_model_and_diffusion(
        **args_to_dict(args, model_and_diffusion_defaults().keys())
    )
    if args.multi_gpu:
        model = th.nn.DataParallel(model,device_ids=[int(id) for id in args.multi_gpu.split(',')])
        model.to(device = th.device('cuda', int(args.gpu_dev)))
    else:
        model.to(dist_util.dev())
    schedule_sampler = create_named_schedule_sampler(args.schedule_sampler, diffusion,  maxt=args.diffusion_steps)


    logger.log("training...")
    TrainLoop(
        model=model,
        diffusion=diffusion,
        classifier=None,
        data=data,
        dataloader=datal,
        batch_size=args.batch_size,
        microbatch=args.microbatch,
        lr=args.lr,
        ema_rate=args.ema_rate,
        log_interval=args.log_interval,
        save_interval=args.save_interval,
        resume_checkpoint=args.resume_checkpoint,
        use_fp16=args.use_fp16,
        fp16_scale_growth=args.fp16_scale_growth,
        schedule_sampler=schedule_sampler,
        weight_decay=args.weight_decay,
        lr_anneal_steps=args.lr_anneal_steps,
    ).run_loop()

def visualize(img):
    _min = img.min()
    _max = img.max()
    normalized_img = (img - _min)/ (_max - _min)
    return normalized_img


def sample():
    dist_util.setup_dist(args)
    logger.configure(dir = args.out_dir)

    if args.data_name == 'ISIC':
        tran_list = [transforms.Resize((args.image_size,args.image_size)), transforms.ToTensor(),]
        transform_test = transforms.Compose(tran_list)

        ds = ISICDataset(args, args.data_dir, transform_test, mode = 'Test')
        args.in_ch = 4
    elif args.data_name == 'BRATS':
        tran_list = [transforms.Resize((args.image_size,args.image_size)),]
        transform_test = transforms.Compose(tran_list)

        ds = BRATSDataset3D(args.data_dir,transform_test)
        args.in_ch = 5
    else:
        tran_list = [transforms.Resize((args.image_size,args.image_size)), transforms.ToTensor()]
        transform_test = transforms.Compose(tran_list)

        ds = CustomDataset(args, args.data_dir, transform_test, mode = 'Test')
        args.in_ch = 4

    datal = th.utils.data.DataLoader(
        ds,
        batch_size=args.batch_size,
        shuffle=True)
    data = iter(datal)

    logger.log("creating model and diffusion...")

    model, diffusion = create_model_and_diffusion(
        **args_to_dict(args, model_and_diffusion_defaults().keys())
    )
    all_images = []


    state_dict = dist_util.load_state_dict(args.model_path, map_location="cpu")
    from collections import OrderedDict
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        # name = k[7:] # remove `module.`
        if 'module.' in k:
            new_state_dict[k[7:]] = v
            # load params
        else:
            new_state_dict = state_dict

    model.load_state_dict(new_state_dict)

    model.to(dist_util.dev())
    if args.use_fp16:
        model.convert_to_fp16()
    model.eval()
    for _ in range(len(data)):
        b, m, path = next(data)  #should return an image from the dataloader "data"
        c = th.randn_like(b[:, :1, ...])
        img = th.cat((b, c), dim=1)     #add a noise channel$
        if args.data_name == 'ISIC':
            slice_ID=path[0].split("_")[-1].split('.')[0]
        elif args.data_name == 'BRATS':
            # slice_ID=path[0].split("_")[2] + "_" + path[0].split("_")[4]
            slice_ID=path[0].split("_")[-3] + "_" + path[0].split("slice")[-1].split('.nii')[0]

        logger.log("sampling...")

        start = th.cuda.Event(enable_timing=True)
        end = th.cuda.Event(enable_timing=True)
        enslist = []

        for i in range(args.num_ensemble):  #this is for the generation of an ensemble of 5 masks.
            model_kwargs = {}
            start.record()
            sample_fn = (
                diffusion.p_sample_loop_known if not args.use_ddim else diffusion.ddim_sample_loop_known
            )
            # pdb.set_trace() #imbedded debug !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            # truncating model output after its internally calculated in dim 2 should fix the issue:
            # https://github.com/SuperMedIntel/MedSegDiff/issues/201
            # fixable by just editing gaussian_diffusion.py:346 to be permissive
            sample, x_noisy, org, cal, cal_out = sample_fn( # broken
                model,
                (args.batch_size, 3, args.image_size, args.image_size), img,
                step = args.diffusion_steps,
                clip_denoised=args.clip_denoised,
                model_kwargs=model_kwargs,
            )

            end.record()
            th.cuda.synchronize()
            print('time for 1 sample', start.elapsed_time(end))  #time measurement for the generation of 1 sample

            co = th.tensor(cal_out)
            if args.version == 'new':
                enslist.append(sample[:,-1,:,:])
            else:
                enslist.append(co)

            if args.debug:
                # print('sample size is',sample.size())
                # print('org size is',org.size())
                # print('cal size is',cal.size())
                if args.data_name == 'ISIC':
                    # s = th.tensor(sample)[:,-1,:,:].unsqueeze(1).repeat(1, 3, 1, 1)
                    o = th.tensor(org)[:,:-1,:,:]
                    c = th.tensor(cal).repeat(1, 3, 1, 1)
                    # co = co.repeat(1, 3, 1, 1)

                    s = sample[:,-1,:,:]
                    b,h,w = s.size()
                    ss = s.clone()
                    ss = ss.view(s.size(0), -1)
                    ss -= ss.min(1, keepdim=True)[0]
                    ss /= ss.max(1, keepdim=True)[0]
                    ss = ss.view(b, h, w)
                    ss = ss.unsqueeze(1).repeat(1, 3, 1, 1)

                    tup = (ss,o,c)
                elif args.data_name == 'BRATS':
                    s = th.tensor(sample)[:,-1,:,:].unsqueeze(1)
                    m = th.tensor(m.to(device = 'cuda:0'))[:,0,:,:].unsqueeze(1)
                    o1 = th.tensor(org)[:,0,:,:].unsqueeze(1)
                    o2 = th.tensor(org)[:,1,:,:].unsqueeze(1)
                    o3 = th.tensor(org)[:,2,:,:].unsqueeze(1)
                    o4 = th.tensor(org)[:,3,:,:].unsqueeze(1)
                    c = th.tensor(cal)

                    tup = (o1/o1.max(),o2/o2.max(),o3/o3.max(),o4/o4.max(),m,s,c,co)

                compose = th.cat(tup,0)
                vutils.save_image(compose, fp = os.path.join(args.out_dir, str(slice_ID)+'_output'+str(i)+".jpg"), nrow = 1, padding = 10)
        ensres = staple(th.stack(enslist,dim=0)).squeeze(0)
        vutils.save_image(ensres, fp = os.path.join(args.out_dir, str(slice_ID)+'_output_ens'+".jpg"), nrow = 1, padding = 10)

In [24]:
train()

Logging to /kaggle/working/out
creating data loader...
creating model and diffusion...
training...
---------------------------
| grad_norm    | 21.9     |
| loss         | 1        |
| loss_cal     | 0.386    |
| loss_cal_q0  | 0.482    |
| loss_cal_q2  | 0.374    |
| loss_cal_q3  | 0.315    |
| loss_diff    | 1        |
| loss_diff_q0 | 1.01     |
| loss_diff_q2 | 0.995    |
| loss_diff_q3 | 1.01     |
| loss_q0      | 1.01     |
| loss_q2      | 0.995    |
| loss_q3      | 1.01     |
| param_norm   | 227      |
| samples      | 4        |
| step         | 0        |
---------------------------
saving model 0...
saving model 0.9999...
---------------------------
| grad_norm    | 16.1     |
| loss         | 0.258    |
| loss_cal     | 0.237    |
| loss_cal_q0  | 0.221    |
| loss_cal_q1  | 0.242    |
| loss_cal_q2  | 0.245    |
| loss_cal_q3  | 0.238    |
| loss_diff    | 0.258    |
| loss_diff_q0 | 0.334    |
| loss_diff_q1 | 0.256    |
| loss_diff_q2 | 0.228    |
| loss_diff_q3 | 0.2

OLD

In [25]:
!ls /kaggle/working/out

emasavedmodel_0.9999_000000.pt	optsavedmodel000000.pt	savedmodel000000.pt
emasavedmodel_0.9999_000004.pt	optsavedmodel000004.pt	savedmodel000004.pt
emasavedmodel_0.9999_000040.pt	optsavedmodel000040.pt	savedmodel000040.pt
emasavedmodel_0.9999_000400.pt	optsavedmodel000400.pt	savedmodel000400.pt
log.txt				progress.csv


In [26]:
%cd /kaggle/working/out
from IPython.display import FileLink
FileLink('/kaggle/working/out/optsavedmodel000000.pt')

/kaggle/working/out


/kaggle/working/out/optsavedmodel000000.pt

In [27]:
!tar -cf models.tar /kaggle/working/out

tar: Removing leading `/' from member names
tar: /kaggle/working/out/models.tar: file is the archive; not dumped


In [28]:
!ls /kaggle/working/out

emasavedmodel_0.9999_000000.pt	models.tar		progress.csv
emasavedmodel_0.9999_000004.pt	optsavedmodel000000.pt	savedmodel000000.pt
emasavedmodel_0.9999_000040.pt	optsavedmodel000004.pt	savedmodel000004.pt
emasavedmodel_0.9999_000400.pt	optsavedmodel000040.pt	savedmodel000040.pt
log.txt				optsavedmodel000400.pt	savedmodel000400.pt


In [29]:
!tar czvf model_tar2.tar.gz /kaggle/working/out

tar: Removing leading `/' from member names
/kaggle/working/out/
/kaggle/working/out/savedmodel000400.pt
/kaggle/working/out/savedmodel000000.pt
/kaggle/working/out/savedmodel000040.pt
/kaggle/working/out/optsavedmodel000004.pt
/kaggle/working/out/optsavedmodel000400.pt
/kaggle/working/out/emasavedmodel_0.9999_000000.pt
/kaggle/working/out/optsavedmodel000040.pt
/kaggle/working/out/model_tar2.tar.gz
tar: /kaggle/working/out/model_tar2.tar.gz: file changed as we read it
/kaggle/working/out/emasavedmodel_0.9999_000040.pt
/kaggle/working/out/optsavedmodel000000.pt
/kaggle/working/out/emasavedmodel_0.9999_000004.pt
/kaggle/working/out/progress.csv
/kaggle/working/out/emasavedmodel_0.9999_000400.pt
/kaggle/working/out/savedmodel000004.pt
/kaggle/working/out/log.txt
/kaggle/working/out/models.tar
tar: /kaggle/working/out: file changed as we read it


In [30]:
!ls -alh

total 27G
drwxr-xr-x 2 root root 4.0K May  3 05:23 .
drwxr-xr-x 4 root root 4.0K May  3 05:09 ..
-rw-r--r-- 1 root root 412M May  3 05:18 emasavedmodel_0.9999_000000.pt
-rw-r--r-- 1 root root 412M May  3 05:16 emasavedmodel_0.9999_000004.pt
-rw-r--r-- 1 root root 412M May  3 05:18 emasavedmodel_0.9999_000040.pt
-rw-r--r-- 1 root root 412M May  3 05:21 emasavedmodel_0.9999_000400.pt
-rw-r--r-- 1 root root 2.4K May  3 05:21 log.txt
-rw-r--r-- 1 root root 6.5G May  3 05:23 models.tar
-rw-r--r-- 1 root root  14G May  3 05:37 model_tar2.tar.gz
-rw-r--r-- 1 root root 823M May  3 05:18 optsavedmodel000000.pt
-rw-r--r-- 1 root root 823M May  3 05:16 optsavedmodel000004.pt
-rw-r--r-- 1 root root 823M May  3 05:18 optsavedmodel000040.pt
-rw-r--r-- 1 root root 823M May  3 05:21 optsavedmodel000400.pt
-rw-r--r-- 1 root root 1.1K May  3 05:21 progress.csv
-rw-r--r-- 1 root root 412M May  3 05:18 savedmodel000000.pt
-rw-r--r-- 1 root root 412M May  3 05:16 savedmodel000004.pt
-rw-r--r-- 1 root root 

In [31]:
!tar -tvf ./model_tar2.tar.gz

drwxr-xr-x root/root         0 2025-05-03 05:22 kaggle/working/out/
-rw-r--r-- root/root 431251992 2025-05-03 05:21 kaggle/working/out/savedmodel000400.pt
-rw-r--r-- root/root 431251992 2025-05-03 05:18 kaggle/working/out/savedmodel000000.pt
-rw-r--r-- root/root 431251992 2025-05-03 05:18 kaggle/working/out/savedmodel000040.pt
-rw-r--r-- root/root 862379209 2025-05-03 05:16 kaggle/working/out/optsavedmodel000004.pt
-rw-r--r-- root/root 862379209 2025-05-03 05:21 kaggle/working/out/optsavedmodel000400.pt
-rw-r--r-- root/root 431251992 2025-05-03 05:18 kaggle/working/out/emasavedmodel_0.9999_000000.pt
-rw-r--r-- root/root 862379209 2025-05-03 05:18 kaggle/working/out/optsavedmodel000040.pt
-rw-r--r-- root/root 3764387840 2025-05-03 05:27 kaggle/working/out/model_tar2.tar.gz
-rw-r--r-- root/root  431251992 2025-05-03 05:18 kaggle/working/out/emasavedmodel_0.9999_000040.pt
-rw-r--r-- root/root  862379209 2025-05-03 05:18 kaggle/working/out/optsavedmodel000000.pt
-rw-r--r-- root/root  43125

In [32]:
!nvidia_smi

/bin/bash: line 1: nvidia_smi: command not found


In [9]:
sample()

Logging to /kaggle/working/out
creating model and diffusion...
sampling...


TypeError: only integer tensors of a single element can be converted to an index

In [ ]:
!ls /kaggle/working/isic/ISBI2016_ISIC_Part1_Test_GroundTruth/

In [ ]:
!ls /kaggle/working/out
from matplotlib import pyplot as plt
from PIL import Image
display(Image.open('/kaggle/working/out/0010016_output_ens.jpg'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_GroundTruth/ISIC_0010016_Segmentation.png'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_Data/ISIC_0010016.jpg'))

display(Image.open('/kaggle/working/out/0010336_output_ens.jpg'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_GroundTruth/ISIC_0010336_Segmentation.png'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_Data/ISIC_0010336.jpg'))

display(Image.open('/kaggle/working/out/0010437_output_ens.jpg'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_GroundTruth/ISIC_0010437_Segmentation.png'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_Data/ISIC_0010437.jpg'))

display(Image.open('/kaggle/working/out/0010494_output_ens.jpg'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_GroundTruth/ISIC_0010494_Segmentation.png'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_Data/ISIC_0010494.jpg'))

display(Image.open('/kaggle/working/out/0010574_output_ens.jpg'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_GroundTruth/ISIC_0010574_Segmentation.png'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_Data/ISIC_0010574.jpg'))